In [138]:
from aequitas.bias import Bias
from aequitas.group import Group
from aequitas.fairness import Fairness
from ml_utils import Pipeline
import pandas as pd
import numpy as np
import logging

In [123]:
pipeline = Pipeline()

chunk = pipeline.load_chunk(chunksize=5000)
data = chunk
max_chunks = 0
while chunk != [] and max_chunks > 0:
    max_chunks = max_chunks - 1
    logger.info("Loading chunk....")
    chunk = pipeline.load_chunk(chunksize=20000)
    data.extend(chunk)
    #logger.info("{} chunks left to load.".format(max_chunks))
columns = [desc[0] for desc in pipeline.db.cur.description]
pipeline.df = pd.DataFrame(data, columns=columns)

columnNumbers = [x for x in range(pipeline.df.shape[1])]  # list of columns' integer indices

columnNumbers.remove(2)  # removing the year column
pipeline.df = pipeline.df.iloc[:, columnNumbers]

pipeline.df['year'] = pd.to_datetime(pipeline.df['year'].apply(str), format='%Y')

 

Connected to evictions DB.


INFO:evictionslog:Connected to evictions DB.


Set schema to 'evictions'.


INFO:evictionslog:Set schema to 'evictions'.


Set background cursor.


INFO:evictionslog:Set background cursor.


In [124]:
pipeline.df.head()

def discretize(data_frame, var, num_bins=4, labels=False):
    new_var= "{}_bins".format(var)
    data_frame[new_var], bins = pd.cut(data_frame[var], bins=num_bins, labels=['low', 'med-low', 'med-high', 'high'], right=True, include_lowest=True, retbins = True)
    return data_frame, bins

In [125]:
pipeline.df.tail()

,state_code,geo_id,name,parent_location,population,poverty_rate,pct_renter_occupied,median_gross_rent,median_household_income,median_property_value,rent_burden,pct_white,pct_af_am,pct_hispanic,pct_am_ind,pct_asian,pct_nh_pi,pct_multiple,pct_other,renter_occupied_households,eviction_filings,evictions,eviction_rate,eviction_filing_rate,imputed,subbed,state,county,tract,testcol,population_pct_change_5yr,poverty_rate_pct_change_5yr,pct_renter_occupied_pct_change_5yr,median_gross_rent_pct_change_5yr,median_household_income_pct_change_5yr,median_property_value_pct_change_5yr,rent_burden_pct_change_5yr,pct_white_pct_change_5yr,pct_af_am_pct_change_5yr,pct_hispanic_pct_change_5yr,pct_am_ind_pct_change_5yr,pct_asian_pct_change_5yr,pct_nh_pi_pct_change_5yr,pct_multiple_pct_change_5yr,pct_other_pct_change_5yr,renter_occupied_households_pct_change_5yr,eviction_filings_pct_change_5yr,evictions_pct_change_5yr,eviction_rate_pct_change_5yr,eviction_filing_rate_pct_change_5yr,conversion_rate,avg_hh_size,population_avg_5yr,poverty_rate_avg_5yr,pct_renter_occupied_5yr,median_gross_rent_avg_5yr,median_household_income_avg_5yr,median_property_value_avg_5yr,rent_burden_avg_5yr,pct_white_avg_5yr,pct_af_am_avg_5yr,pct_hispanic_avg_5yr,pct_am_ind_avg_5yr,pct_asian_avg_5yr,pct_nh_pi_avg_5yr,pct_multiple_avg_5yr,pct_other_avg_5yr,renter_occupied_households_avg_5yr,pct_renter_occupied_avg_5yr,eviction_filings_avg_5yr,evictions_avg_5yr,eviction_rate_avg_5yr,eviction_filing_rate_avg_5yr,population_avg_3yr,poverty_rate_avg_3yr,pct_renter_occupied_3yr,median_gross_rent_avg_3yr,median_household_income_avg_3yr,median_property_value_avg_3yr,rent_burden_avg_3yr,pct_white_avg_3yr,pct_af_am_avg_3yr,pct_hispanic_avg_3yr,pct_am_ind_avg_3yr,pct_asian_avg_3yr,pct_nh_pi_avg_3yr,pct_multiple_avg_3yr,pct_other_avg_3yr,renter_occupied_households_avg_3yr,pct_renter_occupied_avg_3yr,eviction_filings_avg_3yr,evictions_avg_3yr,eviction_rate_avg_3yr,eviction_filing_rate_avg_3yr,renter_occupied_households_pct_change_1yr,pct_renter_occupied_pct_change_1yr,eviction_filings_pct_change_1yr,evictions_pct_change_1yr,eviction_rate_pct_change_1yr,eviction_filing_rate_pct_change_1yr,div_sa,div_enc,population_avg_5yr_tr,poverty_rate_avg_5yr_tr,median_gross_rent_avg_5yr_tr,median_household_income_avg_5yr_tr,median_property_value_avg_5yr_tr,rent_burden_avg_5yr_tr,pct_white_avg_5yr_tr,pct_af_am_avg_5yr_tr,pct_hispanic_avg_5yr_tr,pct_am_ind_avg_5yr_tr,pct_asian_avg_5yr_tr,pct_nh_pi_avg_5yr_tr,pct_multiple_avg_5yr_tr,pct_other_avg_5yr_tr,renter_occupied_households_avg_5yr_tr,pct_renter_occupied_avg_5yr_tr,avg_hh_size_avg_5yr_tr,population_pct_change_5yr_tr,poverty_rate_pct_change_5yr_tr,pct_renter_occupied_pct_change_5yr_tr,median_gross_rent_pct_change_5yr_tr,median_household_income_pct_change_5yr_tr,median_property_value_pct_change_5yr_tr,rent_burden_pct_change_5yr_tr,pct_white_pct_change_5yr_tr,pct_af_am_pct_change_5yr_tr,pct_hispanic_pct_change_5yr_tr,pct_am_ind_pct_change_5yr_tr,pct_asian_pct_change_5yr_tr,pct_nh_pi_pct_change_5yr_tr,pct_multiple_pct_change_5yr_tr,pct_other_pct_change_5yr_tr,renter_occupied_households_pct_change_5yr_tr,avg_hh_size_pct_change_5yr_tr,total_bldg_pct_change_1yr,total_bldg_pct_change_3yr,total_bldg_pct_change_5yr,total_bldg_avg_3yr,total_bldg_avg_5yr,total_units_pct_change_1yr,total_units_pct_change_3yr,total_units_pct_change_5yr,total_units_avg_3yr,total_units_avg_5yr,total_value_pct_change_1yr,total_value_pct_change_3yr,total_value_pct_change_5yr,total_value_avg_3yr,total_value_avg_5yr,conversion_rate_pct_change_1yr,conversion_rate_pct_change_3yr,conversion_rate_pct_change_5yr,conversion_rate_avg_3yr,conversion_rate_avg_5yr,eviction_filings_pct_change_3yr,evictions_pct_change_3yr,eviction_rate_pct_change_3yr,eviction_filing_rate_pct_change_3yr,total_bldg,total_units,total_value,population_tr,poverty_rate_tr,median_gross_rent_tr,median_household_income_tr,median_property_value_tr,rent_burden_tr,pct_white_tr,pct_af_am_tr,pct_hispanic_tr,pct_am_ind_tr,pct_asian

In [157]:
columns = ['pct_renter_occupied', 'pct_white', 'pct_af_am', 'pct_hispanic', 'pct_am_ind', 'pct_asian', 'pct_nh_pi',
    'pct_multiple', 'pct_other', 'renter_occupied_households', 'median_household_income', 'median_property_value']

bin_cols = ['pct_renter_occupied_bins', 'pct_white_bins', 'pct_af_am_bins', 'pct_hispanic_bins', 'pct_am_ind_bins', 'pct_asian_bins', 'pct_nh_pi_bins',
    'pct_multiple_bins', 'pct_other_bins', 'renter_occupied_households_bins', 'median_household_income_bins', 'median_property_value_bins']

bin_cols = ['pct_white_bins', 'renter_occupied_households_bins', 'median_household_income_bins']
mydf = pipeline.df

for col in columns:
    print(col)
    discretize(mydf, col)
    
mydf[bin_cols].tail()

pct_renter_occupied
pct_white
pct_af_am
pct_hispanic
pct_am_ind
pct_asian
pct_nh_pi
pct_multiple
pct_other
renter_occupied_households
median_household_income
median_property_value


,pct_white_bins,renter_occupied_households_bins,median_household_income_bins
4995,med-high,low,low
4996,med-low,low,low
4997,high,low,low
4998,med-low,low,low
4999,med-high,low,low


In [163]:
g = Group()

disc_df = mydf[bin_cols]
disc_df.tail()
disc_df.dtypes
for col in disc_df.columns:
    #print(disc_df[col])
    disc_df[col] = disc_df[col].astype(str)

disc_df['label_value'] = pipeline.df['top20_rate'].astype(str)


s = np.random.random(disc_df.shape[0])
disc_df['score'] = pd.Series(s, )

disc_df.tail()
xtab, _ = g.get_crosstabs(disc_df, score_thresholds={'rank_abs': [0.8]}) #score_thresholds=0.8)
#disc_df.shape

C:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: ht

model_id, score_thresholds 1 {'rank_abs': [0.8]}
COUNTS::: pct_white_bins
high        2742
low          777
med-high     933
med-low      548
dtype: int64
COUNTS::: renter_occupied_households_bins
high           1
low         4973
med-high       1
med-low       25
dtype: int64
COUNTS::: median_household_income_bins
high           4
low         4632
med-high       8
med-low      262
nan           94
dtype: int64


In [164]:
b = Bias()
bdf = b.get_disparity_predefined_groups(xtab, 
                                        {'pct_white_bins':'low',
                                         'renter_occupied_households_bins':'high', 
                                         'median_household_income_bins':'low'})
f = Fairness()
fdf = f.get_group_value_fairness(bdf)

gaf = f.get_group_attribute_fairness(fdf)
gaf

get_disparity_predefined_group()


,model_id,score_threshold,attribute_name,Statistical Parity,Impact Parity,FDR Parity,FPR Parity,FOR Parity,FNR Parity,TPR Parity,TNR Parity,NPV Parity,Precision Parity,TypeI Parity,TypeII Parity,Equalized Odds,Unsupervised Fairness,Supervised Fairness
0,1,0.8_abs,median_household_income_bins,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0.8_abs,pct_white_bins,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,0.8_abs,renter_occupied_households_bins,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
